### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


# Dilepton analysis  (Python) 


This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

In [1]:
import ROOT as R
import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *
%jsroot on

Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/003/3105/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)



Setting luminosity to 10064 pb^-1

###############################
#### Background categories ####
###############################
Category    

## 1. Reading the dataset

Set the analaysis to run (*1largeRjet1lep*, *1lep1tau*, *3lep*, *exactly2lep*, *GamGam*, *2lep*, *4lep*)

Set the directory where you have downloaded the ATLAS OpenData samples you want to run over

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "2lep"

In [3]:
background = R.TChain("mini")
data = R.TChain("mini")

A list of all the background samples, category and their IDs can be found in **Infofile.txt**. The cross-section, efficiencies etc. needed for scaling are stored in the **Files_<---->**. We read these files and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories()
Backgrounds.append("SUSYSlepSlep") 

####################################################################################################
BACKGROUND SAMPLES
####################################################################################################
WARNING 	 File for ggH125_tautaulh not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for VBFH125_tautaulh not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
####################################################################################################
SIGNAL SAMPLES
####################################################################################################
WARNING 	 File for ttH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for ggH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for VBFH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for WpH125J_Wincl_gamgam not found in /storage

In [5]:
MCcat = {}
for cat in allfiles:
    for dsid in allfiles[cat]["dsid"]:
        try:
            MCcat[int(dsid)] = cat
        except:
            continue

In [6]:
dataset_IDs = []
background.Reset()
for b in Backgrounds:
    i = 0
    if not b in mcfiles.keys(): continue
    for mc in mcfiles[b]["files"]:
        if not os.path.isfile(mc): continue
        try:
            dataset_IDs.append(int(mcfiles[b]["dsid"][i]))
            background.Add(mc)
        except:
            print("Could not get DSID for %s. Skipping"%mc)
        i += 1
nen = background.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 118376331 entries for backgrounds


In [7]:
data.Reset(); 
for d in datafiles["data"]["files"]:  
    if not os.path.isfile(d): continue
    data.Add(d)
nen = data.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 24411580 entries for backgrounds


## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) For best dealing with all these histograms we can use dictionaries (Python) or maps (C++). 

In [8]:
hist_VZ = {}; hist_WW = {}; hist_Top = {}; 

In [9]:
for i in dataset_IDs: 
    hist_VZ[i] = R.TH1F() 
    hist_WW[i] = R.TH1F()
    hist_Top[i] = R.TH1F()

In [10]:
for i in dataset_IDs: 
    hist_VZ[i].SetNameTitle("hist_VZ", "Invariant mass, VZ"); 
    hist_WW[i].SetNameTitle("hist_WW", "Invariant mass, WW"); 
    hist_Top[i].SetNameTitle("hist_Top", "Invariant mass, Top");
    hist_VZ[i].SetBins(20,70,110); 
    hist_WW[i].SetBins(20,80,400);
    hist_Top[i].SetBins(20,70,350); 

For data it is only necessary with one histogram for each variable: 

In [11]:
hist_VZ_d = R.TH1F(); 
hist_WW_d = R.TH1F(); 
hist_Top_d = R.TH1F(); 

In [12]:
hist_VZ_d.SetNameTitle("hist_VZ", "Invariant mass, VZ"); 
hist_WW_d.SetNameTitle("hist_WW", "Invariant mass, WW"); 
hist_Top_d.SetNameTitle("hist_Top", "Invariant mass, Top");
hist_VZ_d.SetBins(20,70,110); 
hist_WW_d.SetBins(20,80,400);
hist_Top_d.SetBins(20,70,350); 

In [13]:
# Retrieve lumi from library
%store -r lumi

no stored variable or alias lumi


### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [14]:
for i in dataset_IDs: 
    hist_VZ[i].Reset(); 
    hist_WW[i].Reset(); 
    hist_Top[i].Reset();

In [15]:
hist_VZ_d.Reset(); 
hist_WW_d.Reset(); 
hist_Top_d.Reset(); 

In [16]:
l1 = R.TLorentzVector() 
l2 = R.TLorentzVector() 
dileptons = R.TLorentzVector() 

This is the cell where the analysis is performed. Note that the cell needs to be run twice:

1. with data = 0 to run over MC
2. with data = 1 to run over data

Note that the MC running takes ~5 minutes for 3lep analysis. Much(!!!) more time for e.g. 2lep analysis! Data running is relatively fast for 3lep. 

In [17]:
import pandas as pd
from numpy import sqrt

In [ ]:
%%time
import time
from math import sqrt
isData = 1; 

if isData == 1: ds = data 
else: ds = background
    
columns = {"lep_pt1":[],"lep_eta1":[],"lep_phi1":[],"lep_E1":[], "lep_m1":[],
           "lep_pt2":[],"lep_eta2":[],"lep_phi2":[],"lep_E2":[], "lep_m2":[],
           "jet_n":[], "jet_e":[], "jet_pt": [], "met":[], "nbjet77":[],"nbjet80":[],
           "weight": [], "label": [], "CR" :[]}

i = 0   
count =0
vz = 0
ww = 0
top = 0
for event in ds: 
    
    if i%100000 == 0 and i>0:
        print(f"{100*i/ds.GetEntries():.2f}% completed and {(100000-count)/1000:.3f}% cut.")
        print(f"WW: {ww}, VZ: {vz}, top: {top}")
        count = 0
    i += 1 
    ## Data quality cuts: 
    
    #if i > 10000000: break
    
    #if ds.passGRL == 0: continue
    #if ds.hasGoodVertex == 0: continue
    if(ds.trigM == 0 and ds.trigE == 0):continue  

    ## Event selection:
 
    
    ## Cut #1: Require (exactly) 2 leptons
    if not (ds.lep_n == 2 or ds.lep_n == 3): continue
    
    ## Cut #2: Require opposite charge
    if ds.lep_charge[0] == ds.lep_charge[1]: continue
    
    
    if ds.lep_pt[0]/1000.0 < 25: continue
    if ds.lep_etcone20[0]/ds.lep_pt[0] > 0.15: continue
    if ds.lep_ptcone30[0]/ds.lep_pt[0] > 0.15: continue
    #if not (ds.lep_flag[0] & 512): continue
        
    if ds.lep_pt[1]/1000.0 < 25: continue
    if ds.lep_etcone20[1]/ds.lep_pt[1] > 0.15: continue
    if ds.lep_ptcone30[1]/ds.lep_pt[1] > 0.15: continue
    #if not (ds.lep_flag[1] & 512): continue
    
    
    
    
    ## Set Lorentz vectors: 
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0], ds.lep_phi[0], ds.lep_E[0]/1000.);
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1], ds.lep_phi[1], ds.lep_E[1]/1000.);
    ## Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
    ## to get GeV, which is a more practical and commonly used unit. 
    
    
    
    e = 0
    p = 0
    nr_b = 0
    nbjet77 = 0
    nbjet80 = 0
    for k in range(ds.jet_n):
        e += ds.jet_E[k]
        p += ds.jet_pt[k]
        nr_b += ds.jet_MV2c10[k] < 0.64
        nbjet77 += ds.jet_MV2c10[k] < 0.6459
        nbjet80 += ds.jet_MV2c10[k] < 0.1758
       
    
        
    
    dileptons = l1 + l2;   
    
    #My cuts
    if ds.met_et/1000.0 < 40: continue
    if ds.lep_type[0] == ds.lep_type[1] and dileptons.M() < 121.2: continue
    if nbjet77 > 1: continue
    
    #Cuts from 139f^-1 article. 
    #if ds.lep_type[0] == ds.lep_type[1] and dileptons.M() < 121.2: continue
    #if dileptons.M()<100: continue
    #if ds.met_et/1000.0 < 110: continue
    met_sig = ds.met_et/(1000*sqrt(l1.Et() + l2.Et()))
    #if met_sig < 10: continue 
    #if nr_b > 1: continue
    
    count += 1
    """
    columns["lep_pt1"].append(ds.lep_pt[0]/1000.0)
    columns["lep_eta1"].append(ds.lep_eta[0])
    columns["lep_phi1"].append(ds.lep_phi[0])
    columns["lep_E1"].append(ds.lep_E[0]/1000.0)
    columns["lep_m1"].append(l1.M())
    
    columns["lep_pt2"].append(ds.lep_pt[1]/1000.0)
    columns["lep_eta2"].append(ds.lep_eta[1])
    columns["lep_phi2"].append(ds.lep_phi[1])
    columns["lep_E2"].append(ds.lep_E[1]/1000.0)
    columns["lep_m2"].append(l2.M())
    
    columns["met"].append(ds.met_et/1000.0) 
    
    columns["jet_n"].append(ds.jet_n) 
    columns["jet_e"].append(e/1000)
    columns["jet_pt"].append(p/1000)

    """
    if isData == 0:
        #columns["label"].append(ds.channelNumber)
        W = ((ds.mcWeight)*(ds.scaleFactor_PILEUP)*
             (ds.scaleFactor_ELE)*(ds.scaleFactor_MUON)*
             (ds.scaleFactor_BTAG)*(ds.scaleFactor_LepTRIGGER))*((ds.XSection*lumi)/ds.SumWeights)
        #columns["weight"].append(W)
    #else:
        #columns["label"].append(-999.0)
        #columns["weight"].append(-999.0)
    
    
    #Deine the critical regions.
    filled = False
    if dileptons.M() > 60:
        if ds.lep_type[0] != ds.lep_type[1]:
            #if dileptons.Mt() < 65:
            #if ds.met_et/1000 < 100 and ds.met_et/1000 > 60:
                #if met_sig < 10 and met_sig > 10:
            if dileptons.M() > 100:
                if ds.jet_n ==0:
                    #columns["CR"].append("WW")
                    if data:
                        hist_WW_d.Fill(dileptons.M());
                    else:
                        hist_WW[ds.channelNumber].Fill(dileptons.M(),W);
                    filled = True
                    ww += 1
            #elif dileptons.Mt() > 80:
            #elif ds.met_et/1000 > 110 and met_sig > 10:
            #if dileptons.M() >100:
                if nr_b == 1 and ds.jet_n ==1:
                    #columns["CR"].append("top")
                    if data:
                        hist_Top_d.Fill(dileptons.M());
                    else:
                        hist_Top[ds.channelNumber].Fill(dileptons.M(),W);
                    filled = True
                    top += 1
        #elif dileptons.Mt() > 120:
        #if ds.met_et/1000>110:# and met_sig > 10:
        #if dileptons.M() > 61.2 and dileptons.M()  < 121.2:
        if abs(dileptons.M()-91.2) < 10:
            if ds.jet_n ==0:
                #columns["CR"].append("VZ")
                if data:
                    hist_VZ_d.Fill(dileptons.M());
                else:
                    hist_VZ[ds.channelNumber].Fill(dileptons.M(),W);
                filled = True
                vz += 1
    #if not filled:
        #columns["CR"].append("other")

        
print("Done!")
#df = pd.DataFrame(data=columns)
#df.info

0.41% completed and 99.251% cut.
WW: 65, VZ: 11, top: 61
0.82% completed and 99.303% cut.
WW: 120, VZ: 26, top: 112
1.23% completed and 99.379% cut.
WW: 174, VZ: 37, top: 161
1.64% completed and 99.404% cut.
WW: 217, VZ: 46, top: 203
2.05% completed and 99.473% cut.
WW: 240, VZ: 54, top: 230
2.46% completed and 99.494% cut.
WW: 270, VZ: 56, top: 258
2.87% completed and 99.290% cut.
WW: 327, VZ: 74, top: 315
3.28% completed and 99.152% cut.
WW: 409, VZ: 104, top: 387
3.69% completed and 99.299% cut.
WW: 472, VZ: 117, top: 440
4.10% completed and 99.404% cut.
WW: 518, VZ: 133, top: 475
4.51% completed and 99.328% cut.
WW: 574, VZ: 146, top: 518
4.92% completed and 99.255% cut.
WW: 656, VZ: 168, top: 575
5.33% completed and 99.298% cut.
WW: 738, VZ: 187, top: 617
5.73% completed and 99.349% cut.
WW: 793, VZ: 205, top: 664
6.14% completed and 99.213% cut.
WW: 861, VZ: 228, top: 722
6.55% completed and 99.379% cut.
WW: 915, VZ: 238, top: 777
6.96% completed and 99.391% cut.
WW: 958, VZ: 247

In [ ]:
#df.to_csv("test.csv")

## 3. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 3.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [ ]:
H_VZ = {}; H_WW = {}; H_Top = {}; 

In [ ]:
L = 10.06

In [ ]:
for i in Backgrounds: 
    H_VZ[i] = R.TH1F() 
    H_WW[i] = R.TH1F() 
    H_Top[i] = R.TH1F() 

In [ ]:
for i in Backgrounds: 
    H_VZ[i].SetNameTitle("hist_VZ", "Invariant mass, VZ"); 
    H_WW[i].SetNameTitle("hist_WW", "Invariant mass, WW"); 
    H_Top[i].SetNameTitle("hist_Top", "Invariant mass, Top");
    H_VZ[i].SetBins(20,70,110); 
    H_WW[i].SetBins(20,80,400);
    H_Top[i].SetBins(20,70,350); 

### 3.2 Scale and add histograms 
Now we read our info file, scale all (old) histograms, and then add them to the new histograms we just defined.  

In [ ]:
for i in Backgrounds: 
    H_VZ[i].Reset(); 
    H_WW[i].Reset(); 
    H_Top[i].Reset();

In [ ]:
for dsid in hist_VZ.keys(): 
    
    Type = MCcat[dsid]
    H_VZ[Type].Add(hist_VZ[dsid]); 
    H_WW[Type].Add(hist_WW[dsid]); 
    H_Top[Type].Add(hist_Top[dsid]); 

infofile.close()

### 3.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [ ]:
colors = {}

In [ ]:
colors["Diboson"] = R.kGreen; 
colors["Zjets"] = R.kYellow; 
colors["ttbar"] = R.kRed;
colors["singleTop"] = R.kBlue-7; 
colors["Wjets"] = R.kBlue+3; 
colors["topX"] = R.kOrange+1; 
colors["Higgs"] = R.kMagenta; 
colors["Wjetsincl"] = R.kBlue-10;
colors["Zjetsincl"] = R.kYellow-9;
colors["SUSYSlepSlep"] = R.kYellow-9;

In [ ]:
for h in Backgrounds: 
    H_VZ[h].SetFillColor(colors[h]); 
    H_Top[h].SetFillColor(colors[h]);
    H_WW[h].SetFillColor(colors[h]);
    
    H_VZ[h].SetLineColor(colors[h]); 
    H_Top[h].SetLineColor(colors[h]);
    H_WW[h].SetLineColor(colors[h]);

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.   

In [ ]:
stack_VZ = R.THStack("Invariant mass", "");
stack_Top = R.THStack("Missing ET", ""); 
stack_WW = R.THStack("Lepton pT", ""); 

In [ ]:
for h in Backgrounds: 
    stack_VZ.RecursiveRemove(H_VZ[h]); ## Remove previously stacked histograms  
    stack_Top.RecursiveRemove(H_Top[h]);
    stack_WW.RecursiveRemove(H_WW[h]);
    stack_VZ.Add(H_VZ[h]); 
    stack_Top.Add(H_Top[h]);
    stack_WW.Add(H_WW[h]); 

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [ ]:
R.gStyle.SetLegendBorderSize(0); ## Remove (default) border around legend 
leg = R.TLegend(0.65, 0.60, 0.9, 0.85); 

In [ ]:
leg.Clear();
for i in Backgrounds: 
    leg.AddEntry(H_VZ[i], i, "f")  ## Add your histograms to the legend
leg.AddEntry(hist_VZ_d, "Data", "lep") 

In [ ]:
C = R.TCanvas("c", "c", 600, 600)

In [ ]:
R.gPad.SetLogy() ## Set logarithmic y-axis

In [ ]:
hist_VZ_d.SetLineColor(R.kBlack); 
hist_VZ_d.SetMarkerStyle(R.kFullCircle); 
hist_VZ_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_VZ.Draw("hist"); 
stack_VZ.SetMaximum(1E6); 
stack_VZ.SetMinimum(1); 
stack_VZ.GetYaxis().SetTitle("# events");
stack_VZ.GetYaxis().SetTitleOffset(1.3); 
stack_VZ.GetXaxis().SetTitle("VZ - m_{ll} (GeV)");
stack_VZ.GetXaxis().SetTitleOffset(1.3);
hist_VZ_d.Draw("same E"); 
leg.Draw();
C.Draw();

In [ ]:
hist_Top_d.SetLineColor(R.kBlack); 
hist_Top_d.SetMarkerStyle(R.kFullCircle); 
hist_Top_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_Top.Draw("hist"); 
stack_Top.SetMaximum(1E6); 
stack_Top.SetMinimum(1); 
stack_Top.GetYaxis().SetTitle("# events");
stack_Top.GetYaxis().SetTitleOffset(1.3); 
stack_Top.GetXaxis().SetTitle("Top - m_{ll} (GeV)");
stack_Top.GetXaxis().SetTitleOffset(1.3);
hist_Top_d.Draw("same e"); 
leg.Draw();
C.Draw(); 

In [ ]:
hist_WW_d.SetLineColor(R.kBlack); 
hist_WW_d.SetMarkerStyle(R.kFullCircle); 
hist_WW_d.SetMarkerColor(R.kBlack); 

In [ ]:
stack_WW.Draw("hist"); 
stack_WW.SetMaximum(1E6); 
stack_WW.SetMinimum(1); 
stack_WW.GetYaxis().SetTitle("# events");
stack_WW.GetYaxis().SetTitleOffset(1.3); 
stack_WW.GetXaxis().SetTitle("WW - m_{ll} (GeV)");
stack_WW.GetXaxis().SetTitleOffset(1.3);
hist_WW_d.Draw("same e"); 
leg.Draw();
C.Draw(); 